In [9]:
# DRIVE MOUNT AND ENVIRONMENT VERIFICATION

import os
import sys

# --- Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted.")

# --- Verify project folder exists on Drive ---
PROJECT_ROOT = '/content/drive/MyDrive/kidney-tumour-detection'

required_folders = [
    'dataset/raw',
    'dataset/processed',
    'checkpoints',
    'logs',
    'outputs'
]

print("Verifying folder structure...")
all_good = True
for folder in required_folders:
  full_path = os.path.join(PROJECT_ROOT, folder)
  if os.path.exists(full_path):
    print(f" {folder}")
  else:
    print(f" {folder} missing, creating it now...")
    os.mkdirs(full_path, exist_ok = True)
    print(f" {folder} created!")
    all_good = False

if all_good:
  print("All folders have been verified")
else:
  print("Missing folders have been created")

# --- Check GPU availability ---
import subprocess
print("Checking GPU availability...")
try:
  gpu_info = subprocess.run(
      ['nvidia-smi'],
      capture_output=True,
      text=True
      )
  if gpu_info.returncode == 0:
          for line in gpu_info.stdout.split('\n'):
              if any(x in line for x in ['Tesla', 'A100', 'T4', 'V100', 'L4']):
                  print(f" GPU detected: {line.strip()}")
  else:
    print("nvidia-smi returned an error")
except FileNotFoundError:
    print("No GPU detected - please change runtime type")
    print("Go to: Runtime → Change runtime type → T4 GPU")
except Exception as e:
    print(f" GPU check failed: {e}")

# --- Check RAM ---
import psutil
ram = psutil.virtual_memory()
print(f"RAM available: {ram.available / (1024**3):.1f} gb"
      f"/ {ram.total / (1024**3):.1f} GB total")

# --- Check Runtime Disk Usage ---
disk = psutil.disk_usage('/')
print(f"Runtime disk: {disk.free / (1024**3):.1f} GB free "
      f"/ {disk.total / (1024**3):.1f} GB total")
print("Drive storage: 2TB (psutil cannot read network drives accurately)")
print("Verify manually at drive.google.com")

# --- Check Drive storage ---
drive_disk = psutil.disk_usage(PROJECT_ROOT)
print(f"Drive storage: {drive_disk.free / (1024**3):.1f} GB free "
      f"/ {drive_disk.total / (1024**3):.1f} GB total")

print("\n" + "="*50)
print("Session ready. Project root:", PROJECT_ROOT)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted.
Verifying folder structure...
 dataset/raw
 dataset/processed
 checkpoints
 logs
 outputs
All folders have been verified
Checking GPU availability...
 GPU detected: |   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
RAM available: 11.2 gb/ 12.7 GB total
Runtime disk: 71.1 GB free / 112.6 GB total
Drive storage: 2TB (psutil cannot read network drives accurately)
Verify manually at drive.google.com
Drive storage: 67.5 GB free / 112.6 GB total

Session ready. Project root: /content/drive/MyDrive/kidney-tumour-detection


In [10]:
# GITHUB REPOSITORY SYNC

import os

GITHUB_REPO = "https://github.com/danokundaye/kidney-tumour-detection.git"
REPO_NAME = "kidney-tumour-detection"
CLONE_PATH = f"/content/{REPO_NAME}"

# --- Clone or update repository ---
if os.path.exists(CLONE_PATH):
  print("Repository already exists, pulling latest changes...")
  os.chdir(CLONE_PATH)
  os.system("git pull origin main")
  print("Repository updated")
else:
  print("Cloning repository...")
  os.system(f" git clone {GITHUB_REPO} {CLONE_PATH}")
  print("Repository cloned")

# --- Add repo to Python path so we can import our modules ---
import sys
if CLONE_PATH not in sys.path:
    sys.path.insert(0, CLONE_PATH)
    print(f" Added {CLONE_PATH} to Python path")

print(f"\nWorking directory: {CLONE_PATH}")
print("GitHub sync complete")

Repository already exists, pulling latest changes...
Repository updated

Working directory: /content/kidney-tumour-detection
GitHub sync complete


In [16]:
# INSTALL REQUIRED LIBRARIES

print("Installing required libraries...")
print("This will take 3-5 minutes. Stay calm.")

import subprocess
import sys

def install(package):
  subprocess.check_call(
      [sys.executable, "-m", "pip", "install", package, "-q"],
      stdout=subprocess.DEVNULL,
      stderr=subprocess.DEVNULL
  )

# Verify PyTorch has already been installed
print(" Checking PyTorch...", end=" ")
try:
    import torch
    if torch.cuda.is_available():
        print(f"already installed (v{torch.__version__})")
    else:
        print("Installed but no CUDA - check runtime type")
except ImportError:
    print("Not found, installing...")
    subprocess.check_call(
        [sys.executable, "-m", "pip", "install",
         "torch", "torchvision", "-q"]
    )
    print("Installed")

libraries = [
    ("ultralytics",                               "YOLOv8"),
    ("segmentation-models-pytorch",               "U-Net with ResNet50"),
    ("monai",                                     "Medical imaging utilities"),
    ("nibabel",                                   "NIfTI file reading"),
    ("albumentations",                            "Data augmentation"),
    ("shap",                                      "Explainability"),
    ("opencv-python-headless",                    "Image processing"),
    ("scikit-learn",                              "Metrics"),
    ("matplotlib",                                "Matplotlib"),
    ("seaborn",                                   "Seaborn"),
    ("tqdm",                                      "Progress bars"),
]

for package, name in libraries:
  print(f" Installing {name}...", end=" ")
  try:
    install(package)
    print("installed")
  except Exception as e:
    print(f" Failed: {e}")

print("\nVerifying critical imports...")
verification = {
    "torch":                      "PyTorch",
    "torchvision":                "TorchVision",
    "ultralytics":                "YOLOv8",
    "segmentation_models_pytorch":"U-Net",
    "monai":                      "MONAI",
    "nibabel":                    "NiBabel",
    "albumentations":             "Albumentations",
    "shap":                       "SHAP",
    "cv2":                        "OpenCV",
    "sklearn":                    "Scikit-learn",
}

all_imported = True
for module, name in verification.items():
  try:
    __import__(module)
    print(f" {name}")
  except ImportError:
    print(f" {name} - FAILED TO IMPORT")
    all_imported = False

if all_imported:
    print("\nAll libraries installed and verified")
else:
    print("\nSome libraries failed - rerun this cell")

# Verify PyTorch access to GPU
import torch
print(f"\n PyTorch version: {torch.__version__}")
print(f" CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
else:
    print("CUDA not available - check runtime type")

Installing required libraries...
This will take 3-5 minutes. Stay calm.
 Checking PyTorch... already installed (v2.9.0+cu128)
 Installing YOLOv8... installed
 Installing U-Net with ResNet50... installed
 Installing Medical imaging utilities... installed
 Installing NIfTI file reading... installed
 Installing Data augmentation... installed
 Installing Explainability... installed
 Installing Image processing... installed
 Installing Metrics... installed
 Installing Matplotlib... installed
 Installing Seaborn... installed
 Installing Progress bars... installed

Verifying critical imports...
 PyTorch
 TorchVision
 YOLOv8
 U-Net
 MONAI
 NiBabel
 Albumentations
 SHAP
 OpenCV
 Scikit-learn

All libraries installed and verified

 PyTorch version: 2.9.0+cu128
 CUDA available: True
GPU: Tesla T4
VRAM: 14.6 GB
